In [4]:
import os
import random
import timeit
import pandas as pd
import numpy as np
import tempfile
import tensorflow as tf
import xgboost as xgb
import matplotlib.pyplot as plt
import tensorflow_model_optimization as tfmot
import keras.models as k_models

from scipy.sparse import csr_matrix, save_npz
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import tf_keras as keras
from tf_keras import activations
from tf_keras.models import Model, Sequential, load_model
from tf_keras.layers import Dense, Input, LSTM
from tf_keras.callbacks import EarlyStopping


In [6]:
# LSTM

In [8]:
# Load the model
lstm_model = load_model('LSTM.h5')

In [10]:
df_A = pd.read_csv('Final_EVSE_A.csv')
df_B = pd.read_csv('Final_EVSE_B.csv')

def prepare_categorical_output(y):
    # # Print the `y` matrix before encoding
    # print("y matrix before encoding:\n", y)

    # Convert Series to NumPy array and reshape `y` matrix
    y = y.values.reshape(-1, 1)

    # Print the `y` matrix after reshaping
    # print("y matrix after reshaping:\n", y)
    
    # One-hot encode the target variable
    encoder = OneHotEncoder(sparse_output=False)
    y = encoder.fit_transform(y)

    # Print the `y` matrix after one-hot encoding
    # print("y matrix after one-hot encoding:\n", y)
    
    return y

#Considering B charging station as training and A as testing

def assigning_set(df1, df2):
    # Group by 'CSVNameFile' and split the last 20% of each group into the validation set
    train_list = []
    val_list = []

    grouped = df1.groupby('CSVNameFile')

    for _, group in grouped:
        split_index = int(len(group) * 0.8)
        train_list.append(group.iloc[:split_index])
        val_list.append(group.iloc[split_index:])

    # Concatenate the training and validation sets
    train_df = pd.concat(train_list).reset_index(drop=True)
    val_df = pd.concat(val_list).reset_index(drop=True)

    # Separate features and labels for train and validation sets
    X_train = train_df.drop(columns=['CSVNameFile', 'status', 'multiclass'])
    y_train = prepare_categorical_output(train_df['multiclass'])

    X_val = val_df.drop(columns=['CSVNameFile', 'status', 'multiclass'])
    y_val = prepare_categorical_output(val_df['multiclass'])

    # X_test and y_test from df2 remain unchanged for test evaluation
    X_test = df2.drop(columns=['CSVNameFile', 'status', 'multiclass'])
    y_test = prepare_categorical_output(df2['multiclass'])

    input_dim = X_train.shape[1]
    output_dim = len(np.unique(df1['multiclass']))

    return X_train, X_val, X_test, y_train, y_val, y_test, input_dim, output_dim

X_train, X_val, X_test, y_train, y_val, y_test, input_dim, output_dim = assigning_set(df_B, df_A)

In [14]:
# List of features to keep for pruned model
lstm_features = ['bidirectional_duration_ms', 'src_port', 'dst_port',
                'bidirectional_max_piat_ms', 'dst2src_duration_ms', 
                 'bidirectional_mean_piat_ms', 'bidirectional_bytes', 
                 'src2dst_min_piat_ms', 'bidirectional_min_piat_ms', 'bidirectional_max_ps']

# Selecting specific features from the DataFrames
X_train_pruning = X_train[lstm_features]
X_val_pruning = X_val[lstm_features]
X_test_pruning = X_test[lstm_features]

In [16]:
def create_sequences(X, y, timesteps):
    X_seq, y_seq = [], []
    X = X.values  # Convert DataFrame to Numpy array
    for i in range(len(X) - timesteps):
        X_seq.append(X[i:i + timesteps])
        y_seq.append(y[i + timesteps - 1])
    return np.array(X_seq), np.array(y_seq)

# Generate sequences for training, validation, and testing using the sliding window approach
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

timesteps = 5

X_train, y_train = create_sequences(X_train_pruning, y_train, timesteps)
X_val, y_val = create_sequences(X_val_pruning, y_val, timesteps)
X_test, y_test = create_sequences(X_test_pruning, y_test, timesteps)

print('After using create_sequence')
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)


(1757464, 49) (1757464, 3)
(439382, 49) (439382, 3)
(547854, 49) (547854, 3)
After using create_sequence
(1757459, 5, 10) (1757459, 3)
(439377, 5, 10) (439377, 3)
(547849, 5, 10) (547849, 3)


In [18]:
lstm_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 custom_lstm_1 (LSTM)        (None, 5, 100)            60000     
                                                                 
 custom_lstm_2 (LSTM)        (None, 50)                30200     
                                                                 
 custom_dense (Dense)        (None, 3)                 153       
                                                                 
Total params: 90353 (352.94 KB)
Trainable params: 90353 (352.94 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
# Recreate the LSTM model in Sequential format

# Get input and output shapes of the model
input_dim = X_train.shape[2]  # Get input dimension 
output_dim = lstm_model.output_shape[1]  # Get output dimension

print("Input Dimension:", input_dim)
print("Output Dimension:", output_dim)

new_lstm_model = Sequential([
    Input(shape=(5, input_dim)),  # Input shape is (timesteps, features)
    LSTM(100, return_sequences=True, name='custom_lstm_1'),  # First LSTM layer with 100 units
    LSTM(50, name='custom_lstm_2'),  # Second LSTM layer with 50 units
    Dense(output_dim, activation='softmax', name='custom_dense')  # Final Dense layer with 3 output units
])

new_lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
new_lstm_model.fit(X_train, y_train, validation_data=(X_val, y_val),
                            callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)], 
                            batch_size=32, 
                            epochs=50, verbose=1)

new_lstm_model.save('LSTM_FS.h5')

new_lstm_model.summary()


Input Dimension: 10
Output Dimension: 3
Epoch 1/50


54921/54921 [==============================] - 184s 3ms/step - loss: 0.4840 - accuracy: 0.7432 - val_loss: 0.4839 - val_accuracy: 0.7430
Epoch 2/50
54921/54921 [==============================] - 188s 3ms/step - loss: 0.4831 - accuracy: 0.7436 - val_loss: 0.4843 - val_accuracy: 0.7430
Epoch 3/50
54921/54921 [==============================] - 188s 3ms/step - loss: 0.4830 - accuracy: 0.7437 - val_loss: 0.4842 - val_accuracy: 0.7430
Epoch 4/50
54921/54921 [==============================] - 187s 3ms/step - loss: 0.4829 - accuracy: 0.7437 - val_loss: 0.4842 - val_accuracy: 0.7430
Epoch 5/50
54921/54921 [==============================] - 187s 3ms/step - loss: 0.4830 - accuracy: 0.7437 - val_loss: 0.4846 - val_accuracy: 0.7430
Epoch 6/50
54921/54921 [==============================] - 186s 3ms/step - loss: 0.4829 - accuracy: 0.7437 - val_loss: 0.4841 - val_accuracy: 0.7430
Model: "sequential_2"
________________________________________________

C:\Users\mnoorche\AppData\Local\anaconda3\Lib\site-packages\tf_keras\src\engine\training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [29]:
y_pred = new_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

print(f'Test Accuracy: {accuracy:.4f}')
print(f'Test Precision: {precision:.4f}')
print(f'Test Recall: {recall:.4f}')
print(f'Test F1 Score: {f1:.4f}')


17121/17121 [==============================] - 27s 2ms/step
Test Accuracy: 0.9914
Test Precision: 0.9919
Test Recall: 0.9914
Test F1 Score: 0.9915


In [30]:
# Function to save weights to CSV 

for i, layer in enumerate(new_lstm_model.layers):
    weights = layer.get_weights()
    if weights:
        print(f"Layer {i} - {layer.name}")
        
        # For LSTM layers (which have kernel, recurrent, and bias weights)
        if 'lstm' in layer.name:
            print(f"Weights shape: {weights[0].shape}")  # Kernel weights (input weights)
            print(f"Recurrent weights shape: {weights[1].shape}")  # Recurrent weights
            print(f"Bias shape: {weights[2].shape}")  # Bias

        # For Dense layers (which only have kernel and bias)
        elif 'dense' in layer.name:
            print(f"Weights shape: {weights[0].shape}")  # Kernel weights (input weights)
            print(f"Bias shape: {weights[1].shape}")  # Bias


# Function to save LSTM weights, recurrent weights, and biases to CSV files
def save_lstm_weights_to_csv(model, file_prefix):
    for i, layer in enumerate(model.layers):
        weights = layer.get_weights()
        if weights:  # If the layer has weights (such as LSTM or Dense layers)
            
            # For LSTM layers (which have kernel, recurrent, and bias weights)
            if 'lstm' in layer.name:
                # Save kernel weights (input weights)
                kernel_df = pd.DataFrame(weights[0])  
                kernel_df.to_csv(f"{file_prefix}_layer_{i}_kernel_weights.csv", index=False)
                print(f"Saved kernel weights for layer {i} to {file_prefix}_layer_{i}_kernel_weights.csv")
                
                # Save recurrent weights
                recurrent_df = pd.DataFrame(weights[1])  
                recurrent_df.to_csv(f"{file_prefix}_layer_{i}_recurrent_weights.csv", index=False)
                print(f"Saved recurrent weights for layer {i} to {file_prefix}_layer_{i}_recurrent_weights.csv")
                
                # Save biases
                bias_df = pd.DataFrame(weights[2].reshape(1, -1))  
                bias_df.to_csv(f"{file_prefix}_layer_{i}_biases.csv", index=False)
                print(f"Saved biases for layer {i} to {file_prefix}_layer_{i}_biases.csv")
            
            # For Dense layers (which only have kernel weights and biases)
            elif 'dense' in layer.name:
                # Save kernel weights (input weights)
                kernel_df = pd.DataFrame(weights[0])  
                kernel_df.to_csv(f"{file_prefix}_layer_{i}_kernel_weights.csv", index=False)
                print(f"Saved kernel weights for layer {i} to {file_prefix}_layer_{i}_kernel_weights.csv")
                
                # Save biases
                bias_df = pd.DataFrame(weights[1].reshape(1, -1))  
                bias_df.to_csv(f"{file_prefix}_layer_{i}_biases.csv", index=False)
                print(f"Saved biases for layer {i} to {file_prefix}_layer_{i}_biases.csv")


save_lstm_weights_to_csv(new_lstm_model, 'before_pruning_lstm_FS')



Layer 0 - custom_lstm_1
Weights shape: (10, 400)
Recurrent weights shape: (100, 400)
Bias shape: (400,)
Layer 1 - custom_lstm_2
Weights shape: (100, 200)
Recurrent weights shape: (50, 200)
Bias shape: (200,)
Layer 2 - custom_dense
Weights shape: (50, 3)
Bias shape: (3,)
Saved kernel weights for layer 0 to before_pruning_lstm_FS_layer_0_kernel_weights.csv
Saved recurrent weights for layer 0 to before_pruning_lstm_FS_layer_0_recurrent_weights.csv
Saved biases for layer 0 to before_pruning_lstm_FS_layer_0_biases.csv
Saved kernel weights for layer 1 to before_pruning_lstm_FS_layer_1_kernel_weights.csv
Saved recurrent weights for layer 1 to before_pruning_lstm_FS_layer_1_recurrent_weights.csv
Saved biases for layer 1 to before_pruning_lstm_FS_layer_1_biases.csv
Saved kernel weights for layer 2 to before_pruning_lstm_FS_layer_2_kernel_weights.csv
Saved biases for layer 2 to before_pruning_lstm_FS_layer_2_biases.csv


In [44]:
def pruning(model, X_train, X_val, X_test, y_train, y_val, y_test, final_sparsity, df_name):
    
    # dataset_size = the number of samples in the training set.
    # batch_size = the number of samples processed in one training step.
    # num_epochs = the number of times the entire training set is used to update the model.
    
    # Parameters for the dataset and training
    dataset_size = len(X_train)
    batch_size = 32  
    num_epochs = 20
    
    # Pruning parameters as percentages of the total steps
    start_pct = 0  # Start pruning at the beginning of the training steps
    end_pct = 0.1    # End pruning after 60% of the training steps

    # Step calculations
    steps_per_epoch = dataset_size / batch_size
    total_steps = steps_per_epoch * num_epochs

    start_step = int(total_steps * start_pct)
    end_step = int(total_steps * end_pct)

    # Display the calculated steps
    print(f"Total Steps: {total_steps}")
    print(f"Start Step: {start_step}")
    print(f"End Step: {end_step}")
    
    start_epoch = start_step // steps_per_epoch
    end_epoch = end_step // steps_per_epoch
    print(f"Pruning will start in epoch {int(start_epoch)} and end in epoch {int(end_epoch)}")

    
    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.65,
                                                                 final_sparsity=final_sparsity,
                                                                 begin_step=start_step,
                                                                 end_step=end_step,
                                                                 power=1,
                                                                 frequency=100),
    }

    pruned_model = tfmot.sparsity.keras.prune_low_magnitude(
        model, **pruning_params
    )   

    
    # Compile the pruned model
    pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


    # Set up pruning callbacks and early stopping
    pruning_callbacks = [
        tfmot.sparsity.keras.UpdatePruningStep(),
        tfmot.sparsity.keras.PruningSummaries(log_dir=tempfile.mkdtemp())
    ]

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
      
    callbacks = pruning_callbacks + [early_stopping]

    history = pruned_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=num_epochs, batch_size=batch_size, callbacks=callbacks)

    # Strip pruning wrappers
    final_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

    # Compile the model again after stripping the pruning wrappers
    final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    final_model.save(f'{df_name}.h5') 
    
    # The number of epochs should be greater than num_epochs_for_end_step to make sure pruning is complete
    num_epochs_for_end_step = int(end_step // steps_per_epoch)
    the_number_of_epochs = len(history.history['loss'])
    # Print the number of epochs and the epoch when pruning finished
    print(f"Number of epochs is: {the_number_of_epochs}, and pruning finished at epoch {num_epochs_for_end_step}")
    
    # Check if pruning completed before early stopping
    pruning_completed = the_number_of_epochs > num_epochs_for_end_step
    print(f"Pruning completed before early stopping: {pruning_completed}")

    y_pred = final_model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
    recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
    f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

    print(f'Test Accuracy: {accuracy:.4f}')
    print(f'Test Precision: {precision:.4f}')
    print(f'Test Recall: {recall:.4f}')
    print(f'Test F1 Score: {f1:.4f}')
    
    save_lstm_weights_to_csv(final_model, 'after_pruning_lstm_FS')
  
    return final_model, pruning_completed

In [46]:
the_lstm_model = load_model('LSTM_FS.h5')
PrunedLSTM, pruning_completed = pruning(the_lstm_model, X_train, X_val, X_test, y_train, y_val, y_test, 0.65, 'PrunedLSTM_FS')

Total Steps: 1098411.875
Start Step: 0
End Step: 109841
Pruning will start in epoch 0 and end in epoch 1
Epoch 1/20
54921/54921 [==============================] - 205s 4ms/step - loss: 0.0949 - accuracy: 0.9659 - val_loss: 0.0912 - val_accuracy: 0.9682
Epoch 2/20
54921/54921 [==============================] - 198s 4ms/step - loss: 0.0927 - accuracy: 0.9670 - val_loss: 0.0933 - val_accuracy: 0.9616
Epoch 3/20
54921/54921 [==============================] - 195s 4ms/step - loss: 0.0909 - accuracy: 0.9677 - val_loss: 0.0888 - val_accuracy: 0.9687
Epoch 4/20
54921/54921 [==============================] - 195s 4ms/step - loss: 0.0897 - accuracy: 0.9683 - val_loss: 0.0888 - val_accuracy: 0.9686
Epoch 5/20
54921/54921 [==============================] - 196s 4ms/step - loss: 0.0894 - accuracy: 0.9683 - val_loss: 0.0880 - val_accuracy: 0.9687
Epoch 6/20
54921/54921 [==============================] - 195s 4ms/step - loss: 0.0892 - accuracy: 0.9684 - val_loss: 0.0878 - val_accuracy: 0.9688
Epoch 7

C:\Users\mnoorche\AppData\Local\anaconda3\Lib\site-packages\tf_keras\src\engine\training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Number of epochs is: 11, and pruning finished at epoch 1
Pruning completed before early stopping: True
17121/17121 [==============================] - 28s 2ms/step
Test Accuracy: 0.9921
Test Precision: 0.9925
Test Recall: 0.9921
Test F1 Score: 0.9922
Saved kernel weights for layer 0 to after_pruning_lstm_FS_layer_0_kernel_weights.csv
Saved recurrent weights for layer 0 to after_pruning_lstm_FS_layer_0_recurrent_weights.csv
Saved biases for layer 0 to after_pruning_lstm_FS_layer_0_biases.csv
Saved kernel weights for layer 1 to after_pruning_lstm_FS_layer_1_kernel_weights.csv
Saved recurrent weights for layer 1 to after_pruning_lstm_FS_layer_1_recurrent_weights.csv
Saved biases for layer 1 to after_pruning_lstm_FS_layer_1_biases.csv
Saved kernel weights for layer 2 to after_pruning_lstm_FS_layer_2_kernel_weights.csv
Saved biases for layer 2 to after_pruning_lstm_FS_layer_2_biases.csv


In [47]:
# Function to compare weights before and after pruning, and calculate percentage of zero weights for LSTM
def compare_lstm_weights_before_after(model_before, model_after):
    for layer in model_before.layers:
        # Get the layer name
        layer_name = layer.name
        
        # Get weights and biases from both models
        weights_before = model_before.get_layer(layer_name).get_weights()
        weights_after = model_after.get_layer(layer_name).get_weights()
        
        print(f"Comparing weights for layer: {layer_name}")
        
        # Handle LSTM layers (which have kernel, recurrent, and bias weights)
        if 'lstm' in layer_name:
            # LSTM layers have kernel, recurrent, and biases (some configurations may split biases)
            weight_names = ['Kernel weights', 'Recurrent weights', 'Biases']
        else:
            # Dense layers have just weights and biases
            weight_names = ['Weights', 'Biases']

        # Iterate through kernel, recurrent, and bias weights for LSTM (or just weights and biases for Dense)
        for i, (before, after) in enumerate(zip(weights_before, weights_after)):
            weight_type = weight_names[i]  # Get the appropriate name for the weight type
            
            # Calculate the total number of weights
            total_weights = np.prod(before.shape)
            
            # Calculate the number of weights that have changed to zero
            changes_to_zero = (before != 0) & (after == 0)
            num_changes_to_zero = np.sum(changes_to_zero)
            
            # Calculate the number of zero weights after pruning
            num_zero_weights_after = np.sum(after == 0)
            
            # Calculate the percentage of weights that are now zero
            percentage_zero_weights = (num_zero_weights_after / total_weights) * 100
            
            # Print the results
            print(f"{weight_type} in layer '{layer_name}':")
            print(f"  Total number of weights: {total_weights}")
            print(f"  Weights changed to zero during pruning: {num_changes_to_zero}")
            print(f"  Percentage of zero weights after pruning: {percentage_zero_weights:.2f}%")
            print("")

# Call the function to compare the original LSTM model and the pruned LSTM model
compare_lstm_weights_before_after(new_lstm_model, PrunedLSTM)


Comparing weights for layer: custom_lstm_1
Kernel weights in layer 'custom_lstm_1':
  Total number of weights: 4000
  Weights changed to zero during pruning: 2600
  Percentage of zero weights after pruning: 65.00%

Recurrent weights in layer 'custom_lstm_1':
  Total number of weights: 40000
  Weights changed to zero during pruning: 26000
  Percentage of zero weights after pruning: 65.00%

Biases in layer 'custom_lstm_1':
  Total number of weights: 400
  Weights changed to zero during pruning: 0
  Percentage of zero weights after pruning: 0.50%

Comparing weights for layer: custom_lstm_2
Kernel weights in layer 'custom_lstm_2':
  Total number of weights: 20000
  Weights changed to zero during pruning: 13000
  Percentage of zero weights after pruning: 65.00%

Recurrent weights in layer 'custom_lstm_2':
  Total number of weights: 10000
  Weights changed to zero during pruning: 6500
  Percentage of zero weights after pruning: 65.00%

Biases in layer 'custom_lstm_2':
  Total number of weigh

In [48]:
# Prediction

In [49]:
# Activation functions with overflow protection
def sigmoid(x):
    x = np.clip(x, -88, 88)  # Clipping to prevent overflow in float32
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

def softmax(x):
    exp_x = np.exp(x - np.max(x))  # Numerical stability
    return exp_x / np.sum(exp_x, axis=1)
    

# Function to perform inference with sparse matrix multiplication for LSTM
def sparse_lstm_inference(model, input_data):
    timesteps = input_data.shape[1]  # Get the number of timesteps
    batch_size = input_data.shape[0]
    layer_outputs = input_data

    for layer in model.layers:
        if 'lstm' in layer.name:  # Only focus on LSTM layers
            # Get LSTM weights
            lstm_weights = layer.get_weights()
            kernel_weights = lstm_weights[0]  # Input weights (kernel)
            recurrent_weights = lstm_weights[1]  # Recurrent weights
            bias = lstm_weights[2]  # Biases

            # Split kernel and recurrent weights for gates: input, forget, cell, output
            kernel_i, kernel_f, kernel_c, kernel_o = np.split(kernel_weights, 4, axis=1)
            recurrent_i, recurrent_f, recurrent_c, recurrent_o = np.split(recurrent_weights, 4, axis=1)
            bias_i, bias_f, bias_c, bias_o = np.split(bias, 4)

            # Convert to sparse format for matrix multiplication
            kernel_i, kernel_f, kernel_c, kernel_o = csr_matrix(kernel_i), csr_matrix(kernel_f), csr_matrix(kernel_c), csr_matrix(kernel_o)
            recurrent_i, recurrent_f, recurrent_c, recurrent_o = csr_matrix(recurrent_i), csr_matrix(recurrent_f), csr_matrix(recurrent_c), csr_matrix(recurrent_o)

            # Initialize cell state and hidden state for this LSTM layer based on lstm units in this layer
            lstm_units = kernel_i.shape[1]  # The number of LSTM units for this layer

            # Initialize cell state and hidden state for each sample in the batch
            cell_state = np.zeros((batch_size, lstm_units))  # Initialize cell state (Once per layer, per batch)
            hidden_state = np.zeros((batch_size, lstm_units))  # Initialize hidden state (Once per layer, per batch)
            cell_state = csr_matrix(cell_state)
            hidden_state = csr_matrix(hidden_state)
            
            if len(layer_outputs.shape) == 3:  # First LSTM layer, process timesteps (3D input)
                timestep_hidden_states = np.zeros((batch_size, timesteps, lstm_units))
                
                for t in range(timesteps):
                    current_input = layer_outputs[:, t, :]  # Input for timestep t

                    current_input = csr_matrix(current_input)
                    
                    # Perform matrix multiplications for each gate
                    input_gate = sigmoid(current_input @ kernel_i + hidden_state @ recurrent_i + bias_i)
                    forget_gate = sigmoid(current_input @ kernel_f + hidden_state @ recurrent_f + bias_f)
                    cell_gate = tanh(current_input @ kernel_c + hidden_state @ recurrent_c + bias_c)
                    output_gate = sigmoid(current_input @ kernel_o + hidden_state @ recurrent_o + bias_o)

                    input_gate = csr_matrix(input_gate)
                    forget_gate = csr_matrix(forget_gate)
                    cell_gate = csr_matrix(cell_gate)
                    output_gate = csr_matrix(output_gate)

                    # Update cell state and hidden state
                    # cell_state = forget_gate * cell_state + input_gate * cell_gate
                    # hidden_state = output_gate * tanh(cell_state)
                    cell_state = csr_matrix.multiply(forget_gate, cell_state) + csr_matrix.multiply(input_gate, cell_gate)
                    hidden_state = csr_matrix.multiply(output_gate, tanh(cell_state))

                    # Store hidden state for this timestep
                    timestep_hidden_states[:, t, :] = hidden_state.toarray()

                # After processing all timesteps, pass the entire sequence to the next layer
                layer_outputs = timestep_hidden_states if layer.return_sequences else hidden_state

            else:  # For subsequent LSTM layers, input is 2D (batch_size, lstm_units)
                current_input_sparse = csr_matrix(layer_outputs)  # Convert the input to sparse

                # Perform matrix multiplications for each gate
                input_gate = sigmoid(current_input_sparse @ kernel_i + hidden_state @ recurrent_i + bias_i)
                forget_gate = sigmoid(current_input_sparse @ kernel_f + hidden_state @ recurrent_f + bias_f)
                cell_gate = tanh(current_input_sparse @ kernel_c + hidden_state @ recurrent_c + bias_c)
                output_gate = sigmoid(current_input_sparse @ kernel_o + hidden_state @ recurrent_o + bias_o)

                # Update cell state and hidden state
                # cell_state = forget_gate * cell_state + input_gate * cell_gate
                # hidden_state = output_gate * tanh(cell_state)
                cell_state = csr_matrix.multiply(forget_gate, cell_state) + csr_matrix.multiply(input_gate, cell_gate)
                hidden_state = csr_matrix.multiply(output_gate, tanh(cell_state))
                

                # Pass only the last hidden state to the next layer
                layer_outputs = hidden_state

        elif 'dense' in layer.name:  # Handle dense layers
            dense_weights = layer.get_weights()[0]  # Get dense weights
            bias = layer.get_weights()[1]  # Get biases

            # Convert sparse weights to dense format for multiplication
            sparse_weights = csr_matrix(dense_weights)
            layer_outputs = csr_matrix(layer_outputs)
            
            # Perform matrix multiplication (converting sparse weights to dense)
            layer_outputs = layer_outputs @ sparse_weights
            
            # Add bias to each output row
            layer_outputs = layer_outputs + bias

            # Apply the softmax activation function after the dense layer
            if layer.activation.__name__ == 'softmax':
                layer_outputs = softmax(layer_outputs)

    return layer_outputs


In [50]:
# Example input for inference
first_sample = X_test[0]
first_sample_batch = np.expand_dims(first_sample, axis=0)  # This will make the shape (1, 5, 10)

# Perform inference using the LSTM sparse matrix multiplication
sparse_inference_output = sparse_lstm_inference(PrunedLSTM, first_sample_batch)
print("LSTM sparse inference output:", sparse_inference_output)

# Real prediction for comparison
predict_original_inference_output = PrunedLSTM.predict(first_sample_batch)
print('Prediction of the original LSTM using predict method:', predict_original_inference_output)

LSTM sparse inference output: [[3.04510360e-03 3.50984628e-08 9.96954861e-01]]
1/1 [==============================] - 0s 12ms/step
Prediction of the original LSTM using predict method: [[3.0451023e-03 3.5098427e-08 9.9695492e-01]]


In [51]:
def measure_inference_time(model, pruned_model, X_test):
    # Lists to store inference times for both models
    model_time_list = []
    pruned_model_time_list = []

    # Model before pruning (original model)
    for sample in X_test:
        start_time = timeit.default_timer()
        model.predict(np.expand_dims(sample, axis=0), verbose=False)
        end_time = timeit.default_timer()
        model_time_list.append((end_time - start_time) * 1000)  # Convert to milliseconds
    
    avg_inference_time_model = np.mean(model_time_list)
    print(f"Average inference time per sample for Model before pruning: {avg_inference_time_model:.6f} milliseconds")

    # Model after pruning (pruned model)
    for sample in X_test:
        start_time = timeit.default_timer()
        sparse_lstm_inference(pruned_model, np.expand_dims(sample, axis=0))
        end_time = timeit.default_timer()
        pruned_model_time_list.append((end_time - start_time) * 1000)  # Convert to milliseconds
    
    avg_inference_time_pruned_model = np.mean(pruned_model_time_list)
    print(f"Average inference time per sample for Model after pruning: {avg_inference_time_pruned_model:.6f} milliseconds")
   
    # Save the lists as NumPy arrays
    np.save('model_time_list.npy', np.array(model_time_list))
    np.save('pruned_model_time_list.npy', np.array(pruned_model_time_list))
    
    return avg_inference_time_model, avg_inference_time_pruned_model


In [52]:
sample_size = int(0.1 * X_test.shape[0])  # Calculate 10% of the rows
random_sample_set = X_test[np.random.choice(X_test.shape[0], sample_size, replace=False)]
print(f"Subsampled X_test shape: {random_sample_set.shape}")

Subsampled X_test shape: (54784, 5, 10)


In [53]:
# Convert DataFrame to NumPy array
avg_time_model, avg_time_pruned_model = measure_inference_time(new_lstm_model, PrunedLSTM, random_sample_set)

Average inference time per sample for Model before pruning: 36.409803 milliseconds
Average inference time per sample for Model after pruning: 9.948283 milliseconds


In [54]:
# Function to save all weights of the original model in a single .npz file
def save_original_weights_as_npz(model, file_name):
    all_weights = {}
    for i, layer in enumerate(model.layers):
        weights = layer.get_weights()
        if weights:
            # For LSTM layers (which have kernel, recurrent, and bias weights)
            if 'lstm' in layer.name:
                all_weights[f"layer_{i}_kernel_weights"] = weights[0]  # Kernel weights
                all_weights[f"layer_{i}_recurrent_weights"] = weights[1]  # Recurrent weights
                all_weights[f"layer_{i}_biases"] = weights[2]  # Biases
            # For Dense layers (which only have kernel weights and biases)
            elif 'dense' in layer.name:
                all_weights[f"layer_{i}_weights"] = weights[0]  # Kernel weights
                all_weights[f"layer_{i}_biases"] = weights[1]  # Biases

    np.savez(file_name, **all_weights)
    print(f"Saved original model weights to {file_name}.npz")

# Function to save all pruned (sparse) weights in a single .npz file
def save_pruned_weights_as_npz(model, file_name):
    all_sparse_weights = {}
    for i, layer in enumerate(model.layers):
        weights = layer.get_weights()
        if weights:
            # For LSTM layers (which have kernel, recurrent, and bias weights)
            if 'lstm' in layer.name:
                sparse_kernel_weights = csr_matrix(weights[0])  # Convert kernel to sparse
                sparse_recurrent_weights = csr_matrix(weights[1])  # Convert recurrent weights to sparse
                all_sparse_weights[f"layer_{i}_sparse_kernel_weights"] = sparse_kernel_weights
                all_sparse_weights[f"layer_{i}_sparse_recurrent_weights"] = sparse_recurrent_weights
                all_sparse_weights[f"layer_{i}_biases"] = weights[2]  # Save biases as dense

            # For Dense layers (which only have kernel weights and biases)
            elif 'dense' in layer.name:
                sparse_weights = csr_matrix(weights[0])  # Convert kernel to sparse
                all_sparse_weights[f"layer_{i}_sparse_weights"] = sparse_weights
                all_sparse_weights[f"layer_{i}_biases"] = weights[1]  # Save biases as dense

    # Save the sparse weights in .npz format
    np.savez(file_name, **all_sparse_weights)
    print(f"Saved pruned model weights to {file_name}.npz")

# Function to compare the sizes of two files
def compare_file_sizes(file1, file2):
    size1 = os.path.getsize(file1) / 1024  # Convert to KB
    size2 = os.path.getsize(file2) / 1024  # Convert to KB
    print(f"Size of {file1}: {size1:.2f} KB")
    print(f"Size of {file2}: {size2:.2f} KB")

# Save all weights for original and pruned models
save_original_weights_as_npz(new_lstm_model, 'original_model_weights')
save_pruned_weights_as_npz(PrunedLSTM, 'pruned_model_weights')

# Comparing the size of the original and pruned model weight files
compare_file_sizes('original_model_weights.npz', 'pruned_model_weights.npz')


Saved original model weights to original_model_weights.npz
Saved pruned model weights to pruned_model_weights.npz
Size of original_model_weights.npz: 294.13 KB
Size of pruned_model_weights.npz: 210.84 KB
